# Preprocessing
---

In [11]:
# Imports
import pandas as pd
import numpy as np
import re
from datetime import datetime

In [12]:
# Load Data
data = pd.read_csv('../data/raw/NY_SLI_CLEANED.csv')

C:\Users\natha\AppData\Local\Temp\ipykernel_25548\408629005.py:2: DtypeWarning: Columns (4,8,12,14,15,18) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('../data/raw/NY_SLI_CLEANED.csv')


In [16]:
# Explore - List columns
data.columns.tolist()

['Service Line Locality',
 'Street Address',
 'Zip Code',
 'State',
 'Lead Gooseneck, Pigtail or Connector Currently Present',
 'Current Public Side SL Material',
 'Was Public SL Material Ever Previously Lead',
 'Public SL Material Verification Method',
 'Public SL Installation or Replacement Date',
 'Public SL Size',
 'Customer SL Material',
 'Customer SL Material Verification Method',
 'Lead Solder Present',
 'Building Type',
 'POU or POE Treatment Present',
 'Customer SL Installation or Replacement Date',
 'Customer SL Size',
 'SL Category',
 'Note',
 'Location',
 'Customer SL Material Category',
 'Public SL Material Verification Method Category',
 'Current SL Material Category',
 'Lead Connector Present Category',
 'Current Public Side SL Material Category',
 'Was Public SL Material Ever Previously Lead Category',
 'Public SL Installation Year',
 'Public SL Size Category',
 'Public SL Size Numeric',
 'SL Category Cleaned',
 'year']

In [ ]:
# Extract year from mixed date formats
def extract_year(value):
    if pd.isna(value):
        return None
    
    # Convert to string for processing
    value_str = str(value).strip()
    
    # Case 1: Already a 4-digit year
    if value_str.isdigit() and len(value_str) == 4:
        return int(value_str)

    # Case 3: YYYYMMDD format (8 digits)
    if value_str.isdigit() and len(value_str) == 8:
        try:
            year = int(value_str[:4])
            month = int(value_str[4:6])
            day = int(value_str[6:8])
            # Validate it's a reasonable date
            dt = datetime(year, month, day)
            if 1900 <= year <= 2100:
                return year
        except (ValueError, OSError):
            pass  # Not a valid YYYYMMDD, try other formats
    
    # Case 2: Unix timestamp (large integer, positive or negative)
    try:
        num = float(value_str)
        # Unix timestamps are typically 9-10 digits for recent dates
        if abs(num) > 10000000:  # Likely a timestamp
            dt = datetime.fromtimestamp(num)
            return dt.year
    except (ValueError, OSError):
        pass
    
    try:
        # Try parsing with pandas
        dt = pd.to_datetime(value_str, errors='coerce')
        if pd.notna(dt):
            return dt.year
    except:
        pass

    # Dash-separated or slash-separated dates 1999-2000, get first 4 digits
    year_range_match = re.match(r'^(\d{4})-(\d{2,4})$', value_str)
    if year_range_match:
        first_year = int(year_range_match.group(1))
        second_part = year_range_match.group(2)
        
        # If it's abbreviated (e.g., "76" or "77")
        if len(second_part) == 2:
            # Take the first year as the installation year
            return first_year
        else:
            # Full year provided (e.g., "1976-1977")
            return first_year

    
    # Regular Date Formats
    date_formats = [
        "%m/%d/%Y", "%m-%d-%Y", "%Y/%m/%d", "%Y-%m-%d",
        "%d/%m/%Y"
    ]
    for fmt in date_formats:
        try:
            dt = datetime.strptime(value_str, fmt)
            return dt.year
        except ValueError:
            continue
    
    return None  # If nothing works

# Drop empty rows in the date column
# list number of rows
data = data.dropna(subset=['Customer SL Installation or Replacement Date'])

# Drop unknown entries
data = data[~data['Customer SL Installation or Replacement Date'].isin(['Unknown', 'unknown', 'UNK', 'N/A', 'NA', '0'])]

# Filter out negative numbers
data = data[~data['Customer SL Installation or Replacement Date'].astype(str).str.startswith('-')]

# Apply to your dataframe
data['Customer SL Installation or Replacement Date'] = data['Customer SL Installation or Replacement Date'].apply(extract_year)

# Drop rows where year extraction failed
data = data.dropna(subset=['Customer SL Installation or Replacement Date'])
print("Final data shape after year installation fix:", data.shape[0])

# # Save to CSV in data/processed
data.to_csv('../data/processed/NY_SLI_INSTL_YEARS_ADJUSTED.csv', index=False)

377584
377584
377584
377584
317201
